#### This assignment may be worked individually or in pairs. Enter your name/names here:
    

In [17]:
# Viren Velacheri, Cameron Courtney #names here

# Assignment 1: Decision Trees

In this assignment we'll implement the Decision Tree algorithm to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the Diabetic Retinopathy data set, which contains features from the Messidor image set to predict whether an image contains signs of diabetic retinopathy or not. This dataset has `1150` records and `20` attributes (some categorical, some continuous). You can find additional details about the dataset [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

Attribute Information:

0) The binary result of quality assessment. 0 = bad quality 1 = sufficient quality.

1) The binary result of pre-screening, where 1 indicates severe retinal abnormality and 0 its lack. 

2-7) The results of MA detection. Each feature value stand for the number of MAs found at the confidence levels alpha = 0.5, . . . , 1, respectively. 

8-15) contain the same information as 2-7) for exudates. However, as exudates are represented by a set of points rather than the number of pixels constructing the lesions, these features are normalized by dividing the 
number of lesions with the diameter of the ROI to compensate different image sizes. 

16) The euclidean distance of the center of the macula and the center of the optic disc to provide important information regarding the patient's condition. This feature is also normalized with the diameter of the ROI.

17) The diameter of the optic disc. 

18) The binary result of the AM/FM-based classification.

19) Class label. 1 = contains signs of Diabetic Retinopathy (Accumulative label for the Messidor classes 1, 2, 3), 0 = no signs of Diabetic Retinopathy.

#### Implementation: 
The function prototypes are given to you, please don't change those. You can add additional helper functions for your convenience. 

*Suggestion:* The dataset is substantially big, for the purpose of easy debugging, work with a subset of the data and test your decision tree implementation on that.

#### Notes:
Parts of this assignment will be **autograded** so a couple of caveats :-
- Entropy is calculated using log with base 2, `math.log2(x)`.
- For continuous features ensure that the threshold value lies exactly between 2 values. For example, if for feature 2 the best split occurs between 10 and 15 then the threshold value will be set as 12.5.
- For binary features [0/1] the threshold value will be 0.5. All values < `thresh_val` go to the left child and all values >= `thresh_val` go to the right child.

In [20]:
# Standard Headers
# You are welcome to add additional headers if you wish
# EXCEPT for scikit-learn... You may NOT use scikit-learn for this assignment!
import pandas as pd
import numpy as np
from math import log2
from random import shuffle

In [21]:
class DataPoint:
    def __str__(self):
        return "< " + str(self.label) + ": " + str(self.features) + " >"
    def __init__(self, label, features):
        self.label = label # the classification label of this data point
        self.features = features

Q1. Read data from a CSV file. Put it into a list of `DataPoints`.

In [22]:
def get_data(filename):
    data = []
#     your code goes here
    df = pd.read_csv(filename, header = None)
    # Simple for loop where we create Data point object
    # from each row in dataframe and add to list of data point
    # objects to be returned at the end
    for index in range(len(df.index)):
      row = df.iloc[index]
      newDataPoint = DataPoint(row[19], row[:19])
      data.append(newDataPoint)
    return data

In [23]:
class TreeNode:
    is_leaf = True          # boolean variable to check if the node is a leaf
    feature_idx = None      # index that identifies the feature
    thresh_val = None       # threshold value that splits the node
    prediction = None       # prediction class (only valid for leaf nodes)
    left_child = None       # left TreeNode (all values < thresh_val)
    right_child = None      # right TreeNode (all values >= thresh_val)
    
    def printTree(self, level=0):    # for debugging purposes
        if self.is_leaf:
            print ('-'*level + 'Leaf Node:      predicts ' + str(self.prediction))
        else:
            print ('-'*level + 'Internal Node:  splits on feature ' 
                   + str(self.feature_idx) + ' with threshold ' + str(self.thresh_val))
            self.left_child.printTree(level+1)
            self.right_child.printTree(level+1)

Q2. Implement the function `make_prediction` that takes the decision tree root and a `DataPoint` instance and returns the prediction label.

In [24]:
def make_prediction(tree_root, data_point):
#     your code goes here
    # helper recursive function called
    return traverseTree(tree_root, data_point)

def traverseTree(tree_root, data_point):
  # base case where if leaf return prediction
  if tree_root.is_leaf:
    return tree_root.prediction
  else:
    # take respective path down tree depending on whether
    # it is greater than or less than threshold value
    if data_point.features[tree_root.feature_idx] < tree_root.thresh_val:
      return traverseTree(tree_root.left_child, data_point)
    else:
      return traverseTree(tree_root.right_child, data_point)

Q3. Implement the function `split_dataset` given an input data set, a `feature_idx` and the `threshold` for the feature. `left_split` will have all values < `threshold` and `right_split` will have all values >= `threshold`.

In [25]:
def split_dataset(data, feature_idx, threshold):
    left_split = []
    right_split = []
#     your code goes here
    # simple for loop that splits dataset
    # based on whether it is less than threshold value or not
    # data point is added to respective split list and both lists
    # returned at the end
    for data_point in data:
      if data_point.features[feature_idx] < threshold:
        left_split.append(data_point)
      else:
        right_split.append(data_point)
    return (left_split, right_split)

Q4. Implement the function `calc_entropy` to return the entropy of the input dataset.

In [26]:
def calc_entropy(data):
    entropy = 0.0
#     your code goes here
    # implemented the entropy formula from notes
    total_num = len(data)
    number_of_ones = 0
    # simply loop through where we count number of one labels
    # and got number of zeros based on that.
    for item in data:
      if item.label == 1:
        number_of_ones += 1
    number_of_zeros = total_num - number_of_ones
    zero_prob = number_of_zeros/total_num
    ones_prob = number_of_ones/total_num
    # Do these below checks to prevent log of zero errors from occuring
    if zero_prob == 0:
      zero_prob = 1
    
    if ones_prob == 0:
      ones_prob = 1
    # entropy formula 
    entropy = -(zero_prob)*log2(zero_prob) + -(ones_prob)*log2(ones_prob)
    return entropy

Q5. Implement the function `calc_best_threshold` which returns the best information gain and the corresponding threshold value for one feature at `feature_idx`.

In [27]:
def calc_best_threshold(data, feature_idx):
    best_info_gain = 0.0
    best_thresh = None
#     your code goes here
    # data sorted
    data.sort(key = lambda x: x.features[feature_idx])
    binary_index = [0, 1, 18] # the binary features 
    parent_entropy = calc_entropy(data)
    # if binary feature, use 0.5 threshold and split based on that
    # the info gain is calculated based on formula discussed in class using
    # entropy calculations
    # if not binary features, go through each feature in order and calculate 
    # midpoint or threshold to see what to split on. Info gain is calculated 
    # the same way. Best info gain and threshold are updated accordingly and
    # returned at the end
    if feature_idx in binary_index:
      best_thresh = 0.5
      left_split, right_split = split_dataset(data, feature_idx, best_thresh)
      if len(left_split) != 0:
        parent_entropy -= calc_entropy(left_split) * (len(left_split)/len(data))

      if len(right_split) != 0:
        parent_entropy -= calc_entropy(right_split) * (len(right_split)/len(data))
      best_info_gain = parent_entropy
    else:
      index = 1
      while index < len(data):
        cur_parent = parent_entropy
        if data[index].features[feature_idx] != data[index - 1].features[feature_idx]:
          thresh = (data[index].features[feature_idx] + data[index - 1].features[feature_idx]) / 2
          left_split, right_split = split_dataset(data, feature_idx, thresh)
          if len(left_split) != 0:
            cur_parent -= calc_entropy(left_split) * (len(left_split)/len(data))
          if len(right_split) != 0:
            cur_parent -= calc_entropy(right_split) * (len(right_split)/len(data))
          info_gain = cur_parent
          if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_thresh = thresh
        index += 1
    return (best_info_gain, best_thresh)

Q6. Implement the function `identify_best_split` which returns the best feature to split on for an input dataset, and also returns the corresponding threshold value.

In [28]:
def identify_best_split(data):
    if len(data) < 2:
      return (None, None)
    best_feature = None
    best_thresh = None
#     your code goes here
    best_feature_gain = 0
    number_of_features = 19
    # Go through all features to identify the best feature to split on and 
    # return that and the corresponding threshold
    for feature in range(number_of_features):
      best_info_gain, best_thresh_of_feature = calc_best_threshold(data, feature)
      if best_info_gain > best_feature_gain:
        best_feature_gain = best_info_gain
        best_thresh = best_thresh_of_feature
        best_feature = feature

    return (best_feature, best_thresh)

Q7. Implement the function `create_leaf_node` which returns a `TreeNode` with `is_leaf=True` and `prediction` set to whichever classification occurs most in the dataset at this node.

In [29]:
def create_leaf_node(data):
#     your code goes here
    total_num_threshold = len(data)/2
    result_Tree_Node = TreeNode()
    result_Tree_Node.is_leaf = True
    result_Tree_Node.prediction = 0
    number_of_ones = 0
    # simple for loop to count all one labels.
    # If 50 percent or more labels are ones, then
    # prediction of created node is 1. Otherwise, it is 
    # 0.
    for item in data:
      if item.label == 1:
        number_of_ones += 1
    
    if number_of_ones >= total_num_threshold:
      result_Tree_Node.prediction = 1

    return result_Tree_Node

Q8. Implement the `create_decision_tree` function. `max_levels` denotes the maximum height of the tree (for example if `max_levels = 1` then the decision tree will only contain the leaf node at the root. [Hint: this is where the recursion happens.]

In [30]:
def create_decision_tree(data, max_levels):
#     your code goes here
    # base case where is levels is 1, down to last node so just create it with
    # data
    if max_levels == 1:
      return create_leaf_node(data)
    else:
      best_feature, best_thresh = identify_best_split(data)

      # If none, just create leaf node
      if best_feature is None:
        return create_leaf_node(data)

      # root node created and corresponding best feature and threshold set
      # False is set for it not being leaf and then recursive calls are made
      # to create left and right trees before returning root node
      rootNode = TreeNode()
      rootNode.feature_idx = best_feature
      rootNode.thresh_val = best_thresh
      rootNode.is_leaf = False
      left, right = split_dataset(data, best_feature, best_thresh)
      rootNode.left_child = create_decision_tree(left, max_levels - 1)
      rootNode.right_child = create_decision_tree(right, max_levels - 1)
      return rootNode

Q9. Given a test set, the function `calc_accuracy` returns the accuracy of the classifier. You'll use the `make_prediction` function for this.

In [31]:
def calc_accuracy(tree_root, data):
#     your code goes here
    num_correct = 0
    # Simple for loop for getting total correctly predicted labels
    # before dividing by total length of data to get accuracy
    for data_point in data:
      prediction = make_prediction(tree_root, data_point)
      if prediction == data_point.label:
        num_correct += 1
    return num_correct/len(data)


Q10. Keeping the `max_levels` parameter as 10, use 5-fold cross validation to measure the accuracy of the model. Print the accuracy of the model.

In [32]:
# edit the code here - this is just a sample to get you started
import time
d = get_data("messidor_features.txt")
k = 5
scores = []
# iterate five times through. a different fifth of dataset is used as
# testing set each time while rest used as training data
for i in range (k):
  lower_bound = int((i / 5) * len(d))
  upper_bound = int(((i + 1) / 5) * len(d))
  train_set = d[0: lower_bound] + d[upper_bound: len(d)]
  test_set = d[lower_bound: upper_bound]
  print ('Training set size:', len(train_set))
  print ('Test set size    :', len(test_set))
  # Create the decision Tree
  start = time.time()
  tree = create_decision_tree(train_set, 10)
  end = time.time()
  print ('Time taken:', end - start)
  # calculate the accuracy of the tree
  accuracy = calc_accuracy(tree, test_set)
  print ('The accuracy on the test set is ', str(accuracy * 100.0))
  scores.append(accuracy)
  # tree.printTree()

avg_score = 1.0 * sum(scores) / len(scores)
print ('The overall accuracy on the test set is ', str(avg_score * 100.0))
#t.printTree()

Training set size: 920
Test set size    : 230
Time taken: 153.73692202568054
The accuracy on the test set is  63.91304347826087
Training set size: 920
Test set size    : 230
Time taken: 132.2780203819275
The accuracy on the test set is  63.04347826086957
Training set size: 920
Test set size    : 230
Time taken: 130.3485860824585
The accuracy on the test set is  66.52173913043478
Training set size: 920
Test set size    : 230
Time taken: 135.16045951843262
The accuracy on the test set is  63.91304347826087
Training set size: 920
Test set size    : 230
Time taken: 137.3281581401825
The accuracy on the test set is  64.34782608695652
The overall accuracy on the test set is  64.34782608695653
